In [ ]:
# -*- coding: utf-8 -*-
"""Lista10.ipynb

In [ ]:
Automatically generated by Colaboratory.

In [ ]:
Original file is located at
    https://colab.research.google.com/drive/1xr2oNSd1V8vndw9DzsDdVBH5_to6qSO7

Lista 10 - Regressão Múltipla

 Introdução


<br>
# -*- coding: utf8<br>
from scipy import stats as ss<br>
import matplotlib.pyplot as plt<br>
from numpy.testing import assert_equal, assert_array_almost_equal<br>
import numpy as np<br>
import pandas as pd<br>
plt.ion()<br>
import seaborn as sns<br>
PARAMETROS_RC = {  # Parâmetros RC para o matplotlib<br>
  'figure.figsize': (12, 8),<br>
  'axes.labelsize': 20,<br>
  'axes.titlesize': 20,<br>
  'legend.fontsize': 20,<br>
  'xtick.labelsize': 20,<br>
  'ytick.labelsize': 20,<br>
  'lines.linewidth': 4,<br>
}<br>
sns.set(<br>
    style="white",<br>
    palette="colorblind",<br>
    rc=PARAMETROS_RC,<br>
)<br>
def despine(ax=None):<br>
    if ax is None:<br>
        ax = plt.gca()<br>
    # Hide the right and top spines<br>
    ax.spines['right'].set_visible(False)<br>
    ax.spines['top'].set_visible(False)<br>
    # Only show ticks on the left and bottom spines<br>
    ax.yaxis.set_ticks_position('left')<br>
    ax.xaxis.set_ticks_position('bottom')<br>
ontinuando da aula passada, vamos agora focar em casos mais específicos de regressão. Nesta aula, vamos estender a ideia de mínimos quadrados e de regressão linear para modelos mais complexos. Para tal, vamos continuar nosso foco nos dados de preços de apartamentos em BH.

 Dados

In [ ]:
Observe como temos 4 possíveis preditores de preço:
1. Área
1. Quartos
1. Suítes
1. Vagas
"""

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/aptosBH.txt', index_col=0)
df.head()


Cada preditor é correlacionado com o preço de alguma forma. No pairplot abaixo, observe a última coluna, onde cada linha corresponde a um preditor.
<br>
sns.pairplot(df, diag_kws={'edgecolor':'k'}, plot_kws={'alpha':0.5, 'edgecolor':'k'})<br>
# Regressão Múltipla

In [ ]:
Para realizar uma regressão múltipla, vamos representar nossos dados na forma matricial. Se $n$ é o número de linhas dos nossos dados (observações) e $f$ é o número de colunas (features/atributos), então os dados podem ser representados por:

In [ ]:
$$
\mathbf{X} = \begin{pmatrix}\mathbf {x} _{1}^{\mathsf {T}}\\\mathbf {x} _{2}^{\mathsf {T}}\\\vdots \\\mathbf {x} _{n}^{\mathsf {T}}\end{pmatrix} =\begin{pmatrix}1&x_{11}&\cdots &x_{1f}\\1&x_{21}&\cdots &x_{2f}\\\vdots &\vdots &\ddots &\vdots \\1&x_{n1}&\cdots &x_{nf}\end{pmatrix}
$$

In [ ]:
Observe como a primeira coluna é toda $1$. A mesma vai capturar o fator constante, intercepto, da regressão linear. O nosso modelo é capturado pela equação:

In [ ]:
$$y_i = \theta_0 x_{i0} + \theta_1 x_{i1}^{1} + \theta_2 x_{i2}^{2} + \cdots + \theta_f x_{if}^{f} + \epsilon_i$$

In [ ]:
Aqui, $x_{i0} = 1$ sempre (por isso usamos uma coluna de 1s). Dessa forma, a equação acima pode ser simplificada para:

In [ ]:
$$y_i = \theta_0 + \theta_1 x_{i1}^{1} + \theta_2 x_{i2}^{2} + \cdots + \theta_f x_{if}^{f} + \epsilon_i$$

In [ ]:
Vamos organizar uma matriz de fatores explanatórios X usando pandas.
"""

In [ ]:
y = df['preco']
X = df[['area', 'quartos', 'suites', 'vagas']]
X['intercepto'] = 1
X = X[['intercepto', 'area', 'quartos', 'suites', 'vagas']]
X.head()


Temos 216 observações.
<br>
y.shape<br>
 atributos, onde um deles é o intercepto.


In [ ]:
X.shape


X.values retorna a matriz.
<br>
X = X.values<br>
y = y.values # pegar a matrix<br>
X<br>
# Modelo

In [ ]:
Se $\mathbf{y}$ é a nossa resposta, então o nosso modelo tem a seguinte forma matricial:

In [ ]:
$$\mathbf {y} =\mathbf{X}{\boldsymbol {\theta }}+{\boldsymbol {\varepsilon }} $$

In [ ]:
Cada observação $y_i$ vai ser capturada pelo modelo linear $y_i = \theta_0 + \theta_1 x_{i1}^{1} + \theta_2 x_{i2}^{2} + \cdots + \theta_f x_{if}^{f} + \epsilon_i$. Basta pensar em operações de matrizes e vetores como somatórios! Cada linha de $\mathbf{X}$ multiplica o vetor de parâmetros $\theta$.

 Soma dos erros quadrados

In [ ]:
A função de perda na forma matricial é:

In [ ]:
$$L({\boldsymbol {\theta }}) = ||\mathbf{y} - \mathbf{X} {\boldsymbol {\theta }}||^2_2$$

In [ ]:
Essa função é obtida através do quadrado da norma de Frobenius (com p=2), cuja fórmula é dada por:

In [ ]:
$$||A||_p = \left(\sum_i \sum_j |a_{i,j}|^p\right)^{\frac{1}{p}}$$

 Jacobiana

In [ ]:
Quando temos várias derivadas de equações na forma matricial, estamos computando a Jacobiana J (vetor de derivadas). Cada elemento do vetor J é uma derivada:

In [ ]:
$$J(\theta) =
  \left(
    \begin{matrix}
    \frac{dL}{d\theta_0} & \frac{dL}{d\theta_1} & ... & \frac{dL}{d\theta_f}
    \end{matrix}
  \right)
$$

In [ ]:
Um fator bastante interessante deste modelo é que todas as derivadas (para cada $\theta_i$) têm a mesma forma. Como temos uma soma de fatores lineares, cada $\theta_i$ vai ter o mesmo formato. Assim:

In [ ]:
$$\frac{dL}{d\theta_j} = -2n^{-1} \sum_{i=1}^{n} \left(y_i - \sum_{j=0} \theta_j x_{ij}\right) x_{ij}$$

In [ ]:
$$\frac{dL}{d\theta_j} = -2n^{-1} \sum_{i=1}^{n} \epsilon_i x_{ij}$$

In [ ]:
A função abaixo computa tal derivada explorando o conceito de vetorização.
"""

In [ ]:
def derivadas_regressao_media_old(theta, X, y):
    return -2 * ((y - X @ theta) * X.T).mean(axis=1)

In [ ]:
def derivadas_regressao(theta, X, y):
    return -2 * ((y - X @ theta) @ X)


## Versão nova da função da média
<br>
def derivadas_regressao_media(theta, X, y):<br>
    return -2 * ((y - X @ theta) @ X) / len(y)<br>
# Gradiente Descendente

In [ ]:
Após definir o modelo, a função de erro e a Jacobiana para a Regressão Múltipla, podemos calcular o gradiente descendente!
"""

In [ ]:
def gd(theta, d_fun, X, y, lambda_=0.0001, tol=0.00001, max_iter=10000):
  theta = theta.copy()
  #print('Iter {}; theta = '.format(0), theta)
  old_err_sq = np.inf
  i = 0
  while True:
    # Computar as derivadas
    grad = d_fun(theta, X, y)
    # Atualizar
    theta_novo = theta - lambda_ * grad

    # Parar quando o erro convergir
    err_sq = ((X.dot(theta) - y) ** 2).mean()
    if np.abs(old_err_sq - err_sq) <= tol:
      break
    theta = theta_novo
    old_err_sq = err_sq
    #print('Iter {}; theta = '.format(i+1), theta)
    i += 1
    if i == max_iter:
      break
  return theta


## Gradiente Descendente Estocástico<br>
O gradiente descendente estocástico também funciona! Entretanto, temos que acertar a taxa de aprendizado.<br>
Um dos problemas do uso recorrente do gradiente descendente estocástico é que não sabemos ainda como acertar nossa taxa. Para tal, geralmente fazemos uso de treino/validação/teste (assunto das próximas aulas).<br>


In [ ]:
def sgd(theta, d_fun, X, y, lambda_=0.001, tol=0.01, max_iter=10000):
  theta = theta.copy()
  #print('Iter {}; alpha, beta = '.format(0), theta)
  old_err_sq = np.inf
  for i in range(max_iter):
    # Escolhe ponto aleatório
    r = np.random.randint(len(y))
    X_r, y_r = X[r], y[r]
    X_r = X_r.reshape(1, len(X_r)) # transforma o vetor linha em matriz

    # Deriva e atualiza
    grad = d_fun(theta, X_r, y_r)
    theta_novo = theta - lambda_ * grad

    #Calcula o erro
    err_sq = ((X.dot(theta) - y) ** 2).mean()
    theta = theta_novo
    if err_sq < tol:
      break
      #print('Iter {}; alpha, beta = '.format(i+1), theta)

In [ ]:
  return theta


## Normalização dos Dados<br>
A normalização dos dados pode ser útil para ajudar o gradiente. O algoritmo funciona sem tal passo, porém é mais chato definir uma taxa de aprendizado sem normalização e a convergência é mais lenta.<br>
Quando as features aparecem com ordens de grandeza muito diferentes (ex: idade entre 0 a 100 e renda mensal entre  800 e 100.000 reais) a função de custo é distorcida, tornando o ponto mínimo difícil de alcançar. Sendo assim, um truque importante é garantir que todos as features estejam em uma escala similar.<br>
Abaixo normalizamos as features dos dados de preços de apartamentos em BH e, em seguida, calculamos o gradiente descendente e o gradiente descendente estocástico.<br>


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/aptosBH.txt', index_col=0)
z_df = (df-df.mean())/df.std()

In [ ]:
y = z_df['preco']
X = z_df[['area', 'quartos', 'suites', 'vagas']]
X['intercepto'] = 1

In [ ]:
X = X.values
y = y.values

In [ ]:
theta = np.ones(5)
theta = gd(theta, derivadas_regressao, X, y)

In [ ]:
print("theta = ", theta)
print()
print(X[0:4])

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/aptosBH.txt', index_col=0)
z_df = (df-df.mean())/df.std()

In [ ]:
y = z_df['preco']
X = z_df[['area', 'quartos', 'suites', 'vagas']]
X['intercepto'] = 1

In [ ]:
X = X.values
y = y.values

In [ ]:
theta = np.ones(5)
theta = sgd(theta, derivadas_regressao, X, y, lambda_=0.002)

In [ ]:
print("theta = ", theta)


Observe como os resultados batem com o scikit learn.
<br>
from sklearn.linear_model import LinearRegression<br>
model = LinearRegression(fit_intercept=True)<br>
model.fit(X, y)<br>
model.coef_<br>
model.intercept_<br>
# Erros e Validação

In [ ]:
Ainda podemos usar o R-quadrado para avaliar a qualidade de uma regressão linear múltipla. A interpretação permanece a mesma: quanto da variância dos dados é capturada pelo modelo.
"""

In [ ]:
def erro(y, X, theta):
  return y - X@theta

In [ ]:
def multiple_r_squared(X, y, theta):
  sse = sum(erro(y, X, theta)**2)
  sst = sum((y - np.mean(y))**2)
  return 1.0 - sse / sst

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/aptosBH.txt', index_col=0)

In [ ]:
y = df['preco']
X = df[['area', 'quartos', 'suites', 'vagas']]
X['intercepto'] = 1

In [ ]:
X = X.values
y = y.values

In [ ]:
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print(model.coef_)
print("R2 sem normalizacao = ", model.score(X, y))

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/aptosBH.txt', index_col=0)
z_df=(df-df.mean())/df.std()

In [ ]:
y = z_df['preco']
X = z_df[['area', 'quartos', 'suites', 'vagas']]
X['intercepto'] = 1

In [ ]:
X = X.values
y = y.values

In [ ]:
model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print(model.coef_)
print("R2 com normalizacao = ", model.score(X, y))


## Bootstrap para entender a importância de fatores<br>
Podemos aplicar um bootstrap para entender a importância dos fatores. Nós repetidamente tomamos um `bootstrap_regression` dos nossos dados e estimamos o vetor $\Theta$ com base nessa amostra. Se o coeficiente correspondente a uma das variáveis independentes não variar muito entre as amostras, podemos ter certeza de que nossa estimativa é relativamente segura. Se o coeficiente variar muito entre as amostras, não podemos ficar confiantes em nossa estimativa. Caso o intervalo capture o zero, o fator não importa do ponto de vista estatístico. Vamos fazer uso de scikit learn.<br>
Vamos ver exemplos nos dados dos apartamentos.<br>


In [ ]:
def bootstrap_regression(X, y, n=10000, size=None):
  if size is None:
    size = len(df)

In [ ]:
  T = np.zeros(shape=(n, X.shape[1]))
  for i in range(n):
    # Gera amostras com reposição
    idx = np.random.choice(len(y), len(y))
    Xb = X[idx]
    yb = y[idx]

    # Fit usando sklearn
    model = LinearRegression(fit_intercept=True)
    model.fit(Xb, yb)
    T[i] = model.coef_
  return T


A seguir, não usamos o scikit learn e eliminamos duplicatas da amostra:
<br>
def bootstrap_regression_gd(X, y, n=10000, size=None):<br>
  if size is None:<br>
    size = len(df)<br>
  T = np.zeros(shape=(n, X.shape[1]))<br>
  for i in range(n):<br>
    if i%400 == 0:<br>
      print("bootstrap iter ", i)<br>
    # Gera amostras com reposição e remove duplicatas<br>
    idx = list(np.random.choice(len(y), len(y)))<br>
    Xb = X[idx]<br>
    yb = y[idx]<br>
    # Fit usando gd<br>
    theta = np.ones(X.shape[1])<br>
    theta = gd(theta, derivadas_regressao, Xb, yb)<br>
    T[i] = theta<br>
  return T<br>
y = z_df['preco'].values<br>
names = ['area', 'quartos', 'suites', 'vagas']<br>
X = z_df[names]<br>
X['intercepto'] = 1<br>
X = X.values<br>
T = bootstrap_regression_gd(X, y)<br>
names = ['area', 'quartos', 'suites', 'vagas', 'intercepto']<br>
def plota_bootstrap_para_iesimo_parametro(T, i):<br>
  fig = plt.figure()<br>
  plt.hist(T[:, i], edgecolor='k')<br>
  plt.title('{} - 95% CI Bootstrap: ({:.2f}, {:.2f})'.format(names[i],<br>
                                                              np.percentile(T[:, i], 2.5),<br>
                                                              np.percentile(T[:, i], 97.5)))<br>
  plt.xlabel(r'$\theta_i$')<br>
  plt.ylabel('# Boot Amostras')<br>
  despine()<br>
  plt.show()<br>
plota_bootstrap_para_iesimo_parametro(T, 0)<br>
#  Exercícios - Outros datasets:

# Carros

In [ ]:
Utilizando a base de dados carros, ``hybrid.csv``, vamos fazer um gradiente descendente para uma regressão linear com múltiplas variáveis. As colunas são definidas da seguinte forma:

In [ ]:
- veículo (vehicle): modelo do carro

In [ ]:
- ano (year): ano de fabricação

In [ ]:
- msrp: preço de varejo em dólar sugerido pelo fabricante em 2013.

In [ ]:
- aceleração (acceleration): taxa de aceleração em km por hora por segundo

In [ ]:
- mpg: economia de combustível em milhas por galão

In [ ]:
- classe (class): a classe do modelo.

In [ ]:
Nosso objetivo será estimar o valor de preço sugerido dos carros a partir dos demais atributos (exluindo o nome do veículo e a classe).
Portanto, teremos a regressão definida pela fórmula:

In [ ]:
$$ Y = X\Theta + \epsilon $$

In [ ]:
onde, Y corresponde à coluna ``msrp`` dos dados, e X corresponde às colunas ``year,acceleration,mpg``.

In [ ]:
Observe a forma dos dados e a correlação entre as variáveis:
"""

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/hybrid.csv')
df.head()

In [ ]:
import seaborn as sns
sns.pairplot(df, diag_kws={'edgecolor':'k'}, plot_kws={'alpha':0.5, 'edgecolor':'k'})


**Exercício 1:** Selecione apenas as colunas que serão utilizadas e normalize os dados para que o gradiente descendente rode sem problemas.
<br>
# SELECIONE AS COLUNAS<br>
X = df[["year", "acceleration", "mpg"]]<br>
y = df["msrp"]<br>
# NORMALIZE OS DADOS<br>
X = (X - X.mean()) / X.std(ddof=1)<br>
y = (y - y.mean()) / y.std(ddof=1)<br>
# ACRESCENTE INTERCEPTO<br>
inter = np.ones(y.size)<br>
X.insert(0, "Intercepto", inter, True)<br>
# EXTRAIA MATRIZES COM .VALUES<br>
X = X.values<br>
y = y.values<br>
*Exercício 2:** Implemente a função de gradiente dos parâmetros da regressão, retornando um array com os valores dos gradientes para cada parâmetro.


In [ ]:
def gradients(theta, X, y):
  # YOUR CODE HERE
  # x : matriz nxm
  # y : array nx1
  # theta : array mx1
  return -2 * ((y - X @ theta) @ X) / len(y)


* Implemente a função de gradiente descendente para os parâmetros da regressão, retornando um array com os valores de alpha e os valores de beta para cada coluna. Você deve usar a função `gradients` definida anteriormente.
<br>
def descent(theta0, X, y, learning_rate=0.005, tolerance=0.0000001):<br>
    oldErr = np.inf<br>
    i = 0<br>
    for count in range(10000):<br>
        grads = gradients(theta0, X, y)<br>
        newTheta = theta0 - learning_rate * grads<br>
        newErr = ((X.dot(newTheta) - y) ** 2).mean()<br>
        if np.abs(oldErr - newErr) <= tolerance:<br>
            break<br>
        theta0 = newTheta<br>
        oldErr = newErr<br>
    return newTheta<br>
_teste_param0 = np.array([ 1000, 1, 1, 1 ])<br>
theta = descent(_teste_param0, X, y)<br>
assert_array_almost_equal(np.round(theta,4), np.round([ 0.00201339, -0.04349112,  0.59055261, -0.23979036],4))<br>
*Exercício 3:** Agora vamos tentar avaliar o modelo de regressão linear obtido com o gradiente descendente. Para isso, implemente a função que calcula o R-quadrado.

In [ ]:
Lembre-se que, para calcular o R-quadrado, você precisa do valor da soma total dos quadrados e da soma dos erros quadrados. Os parâmetros de entrada de cada função são dados.
"""

In [ ]:
def sst(y):
  # YOUR CODE HERE
  return ((y - y.mean()) ** 2).sum()

In [ ]:
def predict(X, theta):
  # YOUR CODE HERE
  return X @ theta.T

In [ ]:
def sse(X, y, theta):
  # YOUR CODE HERE
  return y.T @ y - (y.T @ X @ np.linalg.inv(X.T @ X) @ X.T @ y)

In [ ]:
def rsquared(X, y, theta):
  # YOUR CODE HERE
  return 1 - (sse(X, y, theta) / sst(y))

In [ ]:
rs = rsquared(X, y, theta)
assert_equal(np.round(rs,4), np.round(0.5288887684860548,4))


Se observarmos os dados pelos gráficos que mostram as correlações entre as variáveis, podemos perceber que nem todos possuem uma relação linear. Vamos tentar transformar os dados de um dos atributos dos carros, para que uma regressão linear possa ser aplicada com melhores resultados.<br>
**Exercício 4:** Para isso, tire o logaritmo da variável ```mpg``` e, em seguida, z-normalize os dados (todos as variáveis em X).<br>


In [ ]:
y = df["msrp"]
X = df[["year", "acceleration", "mpg"]]

TIRE O LOG DA VARIAVEL MPG

In [ ]:
aux = X.copy()
for i in range(len(y)):
    aux.iloc[i, 2] = np.log(aux.iloc[i, 2])
X = aux

Z-NORMALIZE OS DADOS

In [ ]:
dfNormalized = pd.DataFrame()
numericColumns = X.select_dtypes(include=np.number).columns
means = X[numericColumns].mean()
stds = X[numericColumns].std()
dfNormalized[numericColumns] = (X[numericColumns] - means) / stds

In [ ]:
X = dfNormalized

ADICIONE O INTERCEPTO

In [ ]:
inter: np.ndarray = np.ones(y.size)
X.insert(0, "Intercepto", inter, True)

DEFINA X e Y UTILIZANDO .VALUES

In [ ]:
X = X.values
y = y.values


* Rode novamente o código do gradiente descendente e verifique se o R-quadrado da regressão melhorou ou piorou após a transformação dos dados.
<br>
_teste_param0 = np.array([ 1000, 1, 1, 1 ])<br>
theta = descent(_teste_param0, X, y)<br>
rs1 = rsquared(X, y, theta)<br>
assert_equal(np.round(rs1,4), np.round(0.5543728866213389,4))<br>
## ATENÇÃO

In [ ]:
Os próximos dois exercícios não possuem correção automática, porém são de vital importância. Ambos os datasets utilizados nos exercícios a seguir são datasets com grande número de variáveis, o que se aproxima de problemas reais. Como a realização de testes com os valores é dificultada devido ao alto número de variáveis, optou-se pela não correção automática.

In [ ]:
Tente encontrar o modelo que melhor se ajusta aos dados, com maior valor de $R^2$. Tente também compreender a interpretação dos coeficientes para cada variável explicativa. Em caso de dúvidas, acione o monitor da disciplina.

# Dataset Reações Químicas - SEM CORREÇÃO AUTOMÁTICA

In [ ]:
Um cientista maluco misturou quantidades (em milímetros cúbicos) aleatórias de diversos reagentes e conseguiu gerar uma determinada quantidade (em milímetros cúbicos) de um produto químico, que embora não sirva para nada, não é tóxico e é muito bonito. O problema é que o cientista não sabe quais reagentes estão, de fato, reagindo e gerando o tal produto. Mais importante, ele não sabe a quantidade que ele tem que colocar de cada reagente para gerar uma dada quantidade do produto. Para descobrir, ele misturou quantidades aleatórias de cada reagente 1000 vezes e anotou a quantidade do produto que foi gerada em cada um desses experimentos. Para encontrar a fórmula mágica tão desejada, ele pediu a sua ajuda. Os experimentos estão descritos nas linhas do arquivo. A quantidade gerada do produto para cada experimento está descrita na linha "q_produto".

In [ ]:
E aí? Será que regressão linear múltipla resolve? O professor sabe que resolve, e tem um modelo com $R^2$ superior a $0.99$.

In [ ]:
* Encontre esse modelo e mostre os coeficientes, a qualidade do ajuste, e os fatores significativos. Você pode utilizar as funções definidas anteriormente, ou utilizar a biblioteca sklearn.
"""

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/chemical_reaction.csv')
df.head()

In [ ]:
from sklearn.linear_model import LinearRegression
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values
modelo = LinearRegression()
modelo.fit(X, y)
print("Coeficientes: {}".format(modelo.coef_))
print("Intercept: {}".format(modelo.intercept_))
print("R^2: {}".format(modelo.score(X, y)))


### Dataset FIFA - SEM CORREÇÃO AUTOMÁTICA<br>
O [FIFA](https://pt.wikipedia.org/wiki/FIFA_(s%C3%A9rie)) é um jogo eletrônico de futebol muito popular em que, parte da diversão, é poder jogar partidas envolvendo jogadores reais. Para simular um jogador real, o jogo descreve-os usando diversos atributos (como poder do chute, velocidade, etc) os quais, de alguma forma, representam em sua habilidade como um todo (*overall*).<br>
* Neste exercício você deve encontrar um modelo linear que explique a habilidade *overall* dos jogadores do FIFA a partir das suas outras habilidades. Para isso, utilize o arquivo contendo as habilidades de 20 mil jogadores. Mostre os coeficientes, a qualidade do ajuste e os fatores significativos.<br>
**Dica:** pode haver mais de um modelo!<br>


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/pedroharaujo/ICD_Docencia/master/skills_and_overall_sample_20k.csv', index_col=0)
df.head()

In [ ]:
numericColumns = df.select_dtypes(include=np.number).columns
dfNumeric = df[numericColumns]
X = dfNumeric.iloc[:, 1:].values
y = dfNumeric.iloc[:, 0].values
modelo = LinearRegression()
modelo.fit(X, y)
print("Coeficientes: {}".format(modelo.coef_))
print("Intercept: {}".format(modelo.intercept_))
print("R^2: {}".format(modelo.score(X, y)))